In [1]:
import supereeg as se
import numpy as np


In [2]:

def z2r(z):

    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)


def _to_log_complex(X):
    """
    Compute the log of the given numpy array.  Store all positive members of the original array in the real component of
    the result and all negative members of the original array in the complex component of the result.
    Parameters
    ----------
    X : numpy array to take the log of
    Returns
    ----------
    log_X_complex : The log of X, stored as complex numbers to keep track of the positive and negative parts
    """
    signX = np.sign(X)
    posX = np.log(np.multiply(signX > 0, X))
    negX = np.log(np.abs(np.multiply(signX < 0, X)))

    negX = np.multiply(0+1j, negX)
    negX.real[np.isnan(negX)] = 0

    return posX + negX

def _to_exp_real(C):
    """
    Inverse of _to_log_complex
    """
    posX = C.real
    negX = C.imag
    return np.exp(posX) - np.exp(negX)

def _set_numerator(n_real, n_imag):
    """
    Internal function for setting the numerator (deals with size mismatches)
    """
    numerator = np.zeros_like(n_real, dtype=np.complex128)
    numerator.real = n_real
    numerator.imag = n_imag

def _recover_model(num, denom, z_transform=False):

    m = np.divide(_to_exp_real(num), np.exp(denom)) #numerator and denominator are in log units
    if z_transform:
        np.fill_diagonal(m, np.inf)
        return m
    else:
        np.fill_diagonal(m, 1)
        return z2r(m)


In [3]:
a = np.array([[1,2,3],[4,5,6],[7,8,9,]])
b = np.array([[-1,2,2],[-4,5,5],[-7,8,8,]])
c = np.array([[ 0,  4,  5], [ 0, 10, 11],[ 0, 16, 17]])

## check a + b == c

In [4]:
add_log = _to_log_complex(a)

a_log = _to_log_complex(a)
b_log = _to_log_complex(b)
c_log = _to_log_complex(c)

add_log.real = np.logaddexp(a_log.real,b_log.real)
add_log.imag = np.logaddexp(a_log.imag,b_log.imag)

try_add = _to_exp_real(add_log)

np.allclose(try_add, c)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: divide by zero encountered in log
/Library/Python/2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in multiply
/Library/Python/2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: divide by zero encountered in log


True

## check c - a = b

In [5]:
sub_log = _to_log_complex(a)

## multiply a by -1 
neg_a_log = _to_log_complex(-a)

sub_log.real = np.logaddexp(c_log.real, neg_a_log.real)
sub_log.imag = np.logaddexp(c_log.imag, neg_a_log.imag)

b_try = _to_exp_real(sub_log)
np.allclose(b_try, b)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: divide by zero encountered in log
/Library/Python/2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in multiply
/Library/Python/2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: divide by zero encountered in log


True

## make it into a function

In [6]:
def _logsubexp(x,y):
    """
    Subtracts logged arrays
    Parameters
    ----------
    x : Numpy array
        Log complex array
    y : Numpy array
        Log complex array
    Returns
    ----------
    z : Numpy array
        Returns log complex array of y-x
    """
    y = _to_exp_real(y)
    sub_log = _to_log_complex(x)
    neg_y_log = _to_log_complex(-y)
    sub_log.real = np.logaddexp(x.real, neg_y_log.real)
    sub_log.imag = np.logaddexp(x.imag, neg_y_log.imag)
    return sub_log

In [7]:
try_it = _to_exp_real(_logsubexp(c_log,a_log))

/Library/Python/2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in multiply
/Library/Python/2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in multiply
/Library/Python/2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: divide by zero encountered in log


In [8]:
np.allclose(try_it,b)

True